In [ ]:
library(RAnEn)
library(maps)

stopifnot(packageVersion('RAnEn')>="3.2.5")


## Introduction

This article demonstrates how to use the search space functionality within the `RAnEn` package. If you haven't done so, please read [the instructions for basic usage of `RAnEn`](https://weiming-hu.github.io/AnalogsEnsemble/2018/11/04/demo-1-RAnEn-basics.html) first. This article skips the part that has been covered in the previous article.

The classic `AnEn` technique searches for the most similar historical foreasts at its current location. Therefore, only forecasts from the current station/grid point will be traversed and compared. This search style is referred to as the *Independent Search (IS)*. Another possible search style is extended search, which is referred to as *Search Space Extension (SSE)*. It simply indicates that forecasts at nearby stations/grid points are included in the search process. As a result, the search space is significantly larger when using the search space extension.

There are currently two ways to define what nearby locations to be included into the search. Users can set the nearest number of neighbors to be included and/or a distance threshold. The two restraints can be used together.

You will learn how to use these functions:

- `generateAnalogs`

## Data Description

We reuse the data from the [demo 1 AnEn Basics](https://weiming-hu.github.io/AnalogsEnsemble/2018/11/04/demo-1-RAnEn-basics.html). Please refer to the article for a more detailed data description.



In [ ]:
file.name <- 'data-NAM-StateCollege.RData'
# file.name <- 'data-NAM-Denver.RData'
# file.name <- 'data-METAR-CONUS.Rdata'

if (!file.exists(file.name)) {
  cat('Downloading from the data server which might be slow ...\n')
  download.file(url = paste('https://prosecco.geog.psu.edu/', file.name, sep = ''),
                destfile = file.name)
}

load(file.name)
rm(file.name)


## Generate Temperature Forecasts

The process to generate AnEn IS and AnEn SSE forecasts are similar. First, we create a configuration and the following settings are migrated from the independent search tutorial. **Please note the difference when we specify the type of configuration**.

For a full list of avaiable options, please check the document for [RAnEn::generateConfiguration](https://weiming-hu.github.io/AnalogsEnsemble/R/reference/generateConfiguration.html).



In [ ]:
# Create a configuration for SSE
config <- generateConfiguration('extendedSearch')

# This following part is migrated from the independent search as the
# parameter settings are the same.
#

# Set up options
test.start <- 366
test.end <- 372
search.start <- 1
search.end <- 365

# Set up forecasts and time, FLT information
config$forecasts <- forecasts$Data
config$forecast_times <- forecasts$Times
config$flts <- forecasts$FLTs

# Set up observations and time information
config$search_observations <- observations$Data
config$observation_times <- observations$Times

# Set up other parameters
config$num_members <- sqrt(search.end - search.start + 1)
config$verbose <- 3

# Set up predictand id and predictor weights
config$observation_id <- 8
config$weights <- c(1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1)

# Set up test times
config$test_times_compare <- forecasts$Times[test.start:test.end]

# Set up search times
config$search_times_compare <- forecasts$Times[search.start:search.end]

# Set circular variable if there is any
if ('ParameterCirculars' %in% names(forecasts)) {
  config$circulars <- unlist(lapply(forecasts$ParameterCirculars, function(x) {
    return(which(x == forecasts$ParameterNames))}))
}


Now that we have set up the basic parameters, we need to set the parameters for SSE. A list of parameters specific to SSE is shown below:

- forecast_stations_x
- forecast_stations_y
- num_nearest
- distance
- max_num_search_stations
- preserve_search_stations

`forecast_stations_x` and `forecsat_stations_y` specify the locations of stations/grid points. It is assumed that coordinates use a Cartesian grid. No projection is performed within the package so any geographic projection should be done prior to the AnEn computation.

`num_nearest` is the number of neighbors that should be included into SSE. If `distance` is set, a buffer will be created for each station/grid point, and nearby stations/grid points strictly inside this buffer are utilized.

If `distance` is the only constraint of including nearby stations, to avoid including too many stations within the specified distance, a maximum number of nearby stations should be set using `max_num_search_stations`.

`preserve_search_stations` indicates whether to save the search stations in the returned result. The information can be helpful to analyze what stations are used to generate the AnEn results.

Now that we understand the parameters, we set these parameters.



In [ ]:
# Set up coordinates for forecast stations.
# Change the range of x to [-180, 180] for visualization purpose,
#
config$forecast_stations_x <- forecasts$Xs - 360
config$forecast_stations_y <- forecasts$Ys

# Set the number of nearest neighbors to utilize.
# We want to have 9 search stations including the current staion itself.
#
config$num_nearest <- 9

# Let the package return the search stations for each test station
config$preserve_search_stations <- T


Similar to the [RAnEn basic tutorial](https://weiming-hu.github.io/AnalogsEnsemble/2018/11/04/demo-1-RAnEn-basics.html) we can generate AnEn. This might take a while. On a Mac Air, it takes about 3 minutes.



In [ ]:
AnEn <- generateAnalogs(config)


By now we have our AnEn forecasts. Note that we have the search stations stored in the returned result.



In [ ]:
AnEn


Here we print an AnEn at a specific grid point (#10), a specific day (#1), and a specific forecast lead time (#7). Take a look at the rows (members). The second and the third columns show the station and time index in observations where the forecast values are taken from. More information can be found in the documentation for `RAnEn::generateAnalogs`.



In [ ]:
AnEn$analogs[10, 1, 7, , ]


## Verification

Finally, let's extract the AnEn forecast values and the corresponding observations.



In [ ]:
# Collect analog ensemble values
anen <- AnEn$analogs[, , , , 1]

# Collect observations for the test period and the forecasted weather variable
obs <- alignObservations(observations$Data, observations$Times,
                         config$test_times_compare, config$flts,
                         silent = T, show.progress = F)
obs <- obs[config$observation_id, , , ]


Then we can generate the same verification statistics, and see how AnEn is performing. You can also compare this with results from [the AnEn basic tutorial](https://weiming-hu.github.io/AnalogsEnsemble/2018/11/04/demo-1-RAnEn-basics.html), and see how they are different.



In [ ]:
# Generate verification metrics
ret.MAE <- verifyMAE(anen, obs)
ret.RMSE <- verifyRMSE(anen, obs)
ret.Bias <- verifyBias(anen, obs)
ret.RH <- verifyRankHist(anen, obs)

# Let's make some figures
par(mfrow = c(4, 1), mar = c(3, 4.5, 1, 1))
plot(config$flts/3600, ret.MAE$flt, type = 'b', pch = 1, cex = 0.5,
     xlab = '', ylab = 'MAE')
plot(config$flts/3600, ret.RMSE$flt, type = 'b', pch = 1, cex = 0.5,
     xlab = '', ylab = 'RMSE')
plot(config$flts/3600, ret.Bias$flt, type = 'b', pch = 1, cex = 0.5,
     xlab = 'Lead Times (h)', ylab = 'Bias')
barplot(ret.RH$rank, ylab = 'Rank Frequency')


The following code generates a map showing the search stations of a particular test station. Search stations that contain the most similar forecasts are marked differently from the other search stations.



In [ ]:
# Select a test location of which the search locations will be shown.
test.station.index <- 1
day.index <- 1
flt.index <- 7

# Parameters for plot
cex <- 1
offset <- 10
label.size <- .8
title <- 'Search Space Extension'

# Get the selected stations from ensemble members
selected.stations <- unique(
  AnEn$analogs[test.station.index, day.index, flt.index, , 2])

# plot different types of stations together
par(mfrow = c(1, 1))

# Plot all forecast stations
plot(config$forecast_stations_x, config$forecast_stations_y, xlab = 'x', ylab = 'y',
     main = title, pch = 19, cex = cex/2, col = 'grey', asp=1)

# Plot a base map
map('county', col = 'grey', add = T)

# Plot search stations for the current test station
points(config$forecast_stations_x[AnEn$searchStations[, test.station.index]],
       config$forecast_stations_y[AnEn$searchStations[, test.station.index]],
       pch = 19, cex = cex, col = 'green')

# Plot the test station
points(config$forecast_stations_x[test.station.index],
       config$forecast_stations_y[test.station.index],
       pch = 8, cex = cex, col = 'red')

# Plot the search stations that have contribution to the analog ensemble
points(config$forecast_stations_x[selected.stations],
       config$forecast_stations_y[selected.stations],
       cex = cex*1.6, col = 'red')

if (config$distance > 0) {
  coords <- generateCircleCoords(
    x = config$forecast_stations_x[test.station.index],
    y = config$forecast_stations_y[test.station.index],
    radius = config$distance, np = 100)

  points(coords, cex = .1, pch = 19)
}

legend('top', legend = c('grid point', 'search', 'test', 'selected'),
       pch = c(19, 19, 8 , 1), col = c('grey', 'green', 'red', 'red'), horiz = T)